In [ ]:
# CNN익히기_MNIST 손글씨 인식_컨볼류션 신경망
# 컨볼루션(합성곱) 신경망은 입력된 이미지에서 다시 한번 특징을 추출하기 위해 커널(슬라이딩 윈도)를 도입하는 기법.
# 컨볼루션을 만들면 입력 데이터로부터 더욱 정교한 특징을 추출할 수 잇다.


from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint,EarlyStopping

import matplotlib.pyplot as plt
import numpy
import os
import tensorflow as tf

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.random.set_seed(3)

# 데이터 불러오기

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

# 컨볼루션 신경망의 설정
# keras에서 컨볼루션 층을 추가하는 함수는 Conv2D()이다.
# Conv2D에 입력된 4가지 인자들
# 1. 첫 번째 인자: 커널을 몇 개 적용할지 결정. 여러 개의 커널을 적용하면 서로 다른 컨볼루션 여러개 나옴
# 2. 커널 사이즈: 커널의 크기를 정한다 (행, 열)형식으로 정함
# 3. input_shape: dense층과 마찬가지로 맨 처음 층에 입력되는 값 알려줌, (행, 열, 흑백은1 컬러는3)
# 4. activation: 활성화 함수를 정의한다.
# 컨볼루션 층을 통해 이미지 특징을 도출하였지만 여전히 크고 복잡하면 이를 다시 축소해야 한다. 이 과정을 풀링 또는 서브 샘플링이라 한다.
# 풀링 기법에는 정해진 구역 안에서 최댓값을 뽑아내는 max pooling(보편적)과 평균값을 뽑아내는 average pooling등이 있디.
# max pooling은 MaxPooling2D()함수를 사용해서 적용할 수 있다.
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2)) #pool_size는 풀링 창의 크기를 정하는 것으로, 2로 정하면 전체 크기가 절반으로 줄어듦.
model.add(Dropout(0.25)) #과적합을 피하도록 해줌. 랜덤하게 노드를 끔으로써 학습 데이터에 지나치게 치우쳐 학습되는 과적합 방지
model.add(Flatten()) #컨볼루션층이나 맥스풀링층은 이미지를 2차원 배열인 채로 다루고 있어 flatten함수로 1차원으로 바꿔줌.
model.add(Dense(128,  activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 모델 최적화 설정
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

# 모델의 실행
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=30, batch_size=200, verbose=0, callbacks=[early_stopping_callback,checkpointer])

# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))

# 테스트 셋의 오차
y_vloss = history.history['val_loss']

# 학습셋의 오차
y_loss = history.history['loss']

# 그래프로 표현
x_len = numpy.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()



Epoch 1: val_loss improved from inf to 0.05346, saving model to ./model\01-0.0535.hdf5

Epoch 2: val_loss improved from 0.05346 to 0.04075, saving model to ./model\02-0.0408.hdf5

Epoch 3: val_loss improved from 0.04075 to 0.03696, saving model to ./model\03-0.0370.hdf5

Epoch 4: val_loss improved from 0.03696 to 0.03214, saving model to ./model\04-0.0321.hdf5

Epoch 5: val_loss improved from 0.03214 to 0.03076, saving model to ./model\05-0.0308.hdf5

Epoch 6: val_loss improved from 0.03076 to 0.03024, saving model to ./model\06-0.0302.hdf5
